<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/gp_aprendizaje/modeloGausianoLineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias importadas y constantes usadas

In [ ]:
# Modulos y constantes

import numpy as np
import random
import scipy
import scipy.linalg
from matplotlib import pyplot as plt

pi = np.pi

sigma_signal = 0.5 # desviación estándar del ruido usado para enmascarar la señal de entrenamiento

sigma_b = 0.5 # En distribucion gaussiana a priori de modelo de regresióón linear, desviación estándar de la ordenada
sigma_m = 0.5 # En distribucion gaussiana a priori de modelo de regresióón linear, desviación estándar de la pendiente

La siguiente celda contiene informacion sobre el conjunto de puntos - objetivos sobre el que entrenar el proceso gausiano, así como una especificación a priori de la matriz de covarianzas

In [ ]:
n_tp = 50 # puntos de entrenamiento

x_bot_b = 0.
x_top_b = 10.


x = np.linspace(x_bot_b,x_top_b,n_tp)
y = np.fromiter([np.sin(x)+np.cos(x)+sigma_signal*random.random() for x in x],float)

proyeccionlinear = espacioPropio()
mCov,phi_x,mean = proyeccionlinear.linear(x,sigma_b,sigma_m)

print(phi_x.size)

gp_cov_diag = gaussProcess(mCov,np.mean,phi_x,y)

realizacion1 = gp_cov_diag.multivariateGauss(10000)


realizacion1 = datosVisualizacion(x,y,realizacion1[0],realizacion1[1])
realizacion1.mostrarPlot()


100


ValueError: ignored

Este google colab sera un espacio donde practicar conceptos básicos sobre procesos gausianos, en general desarrollar clases para reproducir los conceptos básicos de estos y practicar alguna regresión "de juguete".



In [ ]:
class espacioPropio():
  '''
  Esta clase define el espacio propio donde proyectamos y la matriz de covarianza resultante
  '''

  def linear(self,x, sigma_b,sigma_m):
    '''
    Se interpretan sigma_b y sigma_m como varianzas de distribuciones a priori gaussianas
    para la ordenada en origen y para la pendiente de nuestro modelo linear.
    x es el input
    '''
    return np.array([[sigma_b,0],[0,sigma_m]]), np.array([np.array([1,xe]) for xe in x]),np.zeros(x.size)




class gaussProcess():
  def __init__(self,CovM,Mean, Input, Training):
    '''
    Los objetos de esta clase modelan procesos gaussianos caracterizados por su promedio y covarianza
    '''
    self.input = Input # Dominio de entrenamiento del modelo
    self.training = Training # Objetivos de la regresióón en el modelo
    self.covM = CovM
    self.mean = Mean
    self.cholDescomp()

  def cholDescomp(self):
    '''
    Cholesky decomposition
    '''
    self.L = scipy.linalg.cholesky(np.matmulself.covM, lower=True)

  def multivariateGauss(self,n):
    '''
    Muestrea colecciones de numeros aleatorios siguiente una distribucion 
    '''
    
    # n es la cantidad de numeros gaussianos a generar
    gaussNumbers = np.fromiter([ random.random() for x in range(n) ],float)
    gaussNumbers_dom = np.fromiter([ random.random() for x in range(n) ],float)
    return [self.mean + np.matmul(self.L,gaussNumbers.T)]


class datosVisualizacion():
  '''
  Esta clase contiene los datos de entrenamiento y realizaciones de los procesos gaussianos para la visualización de los mismos
  '''
  def __init__(self,X,Y,X_star,Y_star):
    self.x = X
    self.y = Y
    self.x_star = X_star
    self.y_star = Y_star
    self.inicializacionPlot()

  def realizacion_gp(self,x_star, y_star):
    '''
    Almacena realizacion del proceso gausiano
    '''

  def inicializacionPlot(self):
    '''
    Inicializa el objeto matplotlib
    '''
    self.fig,self.ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    self.ax.plot(self.x,self.y,'go')
    self.ax.plot(self.x_star,self.y_star,'b--')

  def mostrarPlot(self):
    '''
    Muestra plot
    '''
    self.fig.show()



In [ ]:
K = np.array([[2,0,0],[0,5,0],[0,0,6]])
m = np.array([1,2,3])

A = np.array([np.array([1,xe]) for xe in x])

print(A[1])

B = np.array([[1,1],[1,1]])

print(np.matmul(A,B))

#np.fromiter([np.array([1,xe]) for xe in x],float)

#np.array([[0.3,0],[0,0.8]])

[1.         0.20408163]
[[ 1.          1.        ]
 [ 1.20408163  1.20408163]
 [ 1.40816327  1.40816327]
 [ 1.6122449   1.6122449 ]
 [ 1.81632653  1.81632653]
 [ 2.02040816  2.02040816]
 [ 2.2244898   2.2244898 ]
 [ 2.42857143  2.42857143]
 [ 2.63265306  2.63265306]
 [ 2.83673469  2.83673469]
 [ 3.04081633  3.04081633]
 [ 3.24489796  3.24489796]
 [ 3.44897959  3.44897959]
 [ 3.65306122  3.65306122]
 [ 3.85714286  3.85714286]
 [ 4.06122449  4.06122449]
 [ 4.26530612  4.26530612]
 [ 4.46938776  4.46938776]
 [ 4.67346939  4.67346939]
 [ 4.87755102  4.87755102]
 [ 5.08163265  5.08163265]
 [ 5.28571429  5.28571429]
 [ 5.48979592  5.48979592]
 [ 5.69387755  5.69387755]
 [ 5.89795918  5.89795918]
 [ 6.10204082  6.10204082]
 [ 6.30612245  6.30612245]
 [ 6.51020408  6.51020408]
 [ 6.71428571  6.71428571]
 [ 6.91836735  6.91836735]
 [ 7.12244898  7.12244898]
 [ 7.32653061  7.32653061]
 [ 7.53061224  7.53061224]
 [ 7.73469388  7.73469388]
 [ 7.93877551  7.93877551]
 [ 8.14285714  8.14285714]
 [ 8